##### pip install earthengine-api geemap
##### pip install setuptools

In [1]:
import ee
import geemap
import datetime

d:\Pouya Git\advance_computer_vision\NVDI\.venv\Lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
ee.Authenticate()

In [ ]:
# احراز هویت به Google Earth Engine (اولین بار اجرا نیاز به ورود از طریق مرورگر دارد)
ee.Initialize(project='ee-project-461115')

# تعریف منطقه انزلی (نقطه‌ای با شعاع حدود 10 کیلومتر)
anzali_point = ee.Geometry.Point([49.4700, 37.4728])
anzali_region = anzali_point.buffer(10000)  # شعاع به متر

# بارگذاری تصاویر Sentinel-2 برای تابستان 2023 با ابر کمتر از 10٪
start_date = '2023-06-01'
end_date = '2023-09-01'

collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(anzali_region) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median() \
    .clip(anzali_region)

# محاسبه NDVI: (NIR - RED) / (NIR + RED)
ndvi = collection.normalizedDifference(['B8', 'B4']).rename('NDVI')
print('NDVI' , ndvi.getInfo())

# محاسبه NDWI: (GREEN - NIR) / (GREEN + NIR)
ndwi = collection.normalizedDifference(['B3', 'B8']).rename('NDWI')

# محاسبه MNDWI: (GREEN - SWIR1) / (GREEN + SWIR1)
mndwi = collection.normalizedDifference(['B3', 'B11']).rename('MNDWI')

# نمایش نقشه در مرورگر با استفاده از geemap
Map = geemap.Map(center=[37.4728, 49.4700], zoom=11)

Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')
Map.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')
Map.addLayer(mndwi, {'min': -1, 'max': 1, 'palette': ['white', 'cyan']}, 'MNDWI')
Map.addLayer(anzali_region, {}, 'Anzali Region')

Map


NDVI {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [3, 3], 'origin': [48, 36], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[49.47, 37.562788659514815], [49.438292835442866, 37.5592028769008], [49.40911810760921, 37.54873194502907], [49.384803991265116, 37.53221200296216], [49.36728724167959, 37.510961523865845], [49.35795861462284, 37.486675272812235], [49.357553292617666, 37.461288395707875], [49.36609467721929, 37.43682177286631], [49.382895292284964, 37.41522113506815], [49.40661386285674, 37.398202752153004], [49.43536329801121, 37.38711779769351], [49.46686061185352, 37.382845880870065], [49.4986069454941, 37.38572588425232], [49.5280838898616, 37.395529349131934], [49.55295128085091, 37.41147841598188], [49.57123154663176, 37.43230696735139], [49.581466531714305, 37.456360345094645], [49.58283450314634, 37

Map(center=[37.4728, 49.47], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…